<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/Txt2ADiff_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Txt2ADiff Baseline

本notebook旨在从零开始在colab中搭建一个lora和controlnet微调测试的环境

示例工作流参考https://civitai.com/articles/3554/comfyui-txt2adiff

V0.1 created by [Weishi MI](https://github.com/hhhwmws0117)

**注意：**

**本代码在运行时，在创建好Comfy服务后，请首先导入工作流，再使用ComfyUI-Manager对缺失的节点进行安装，安装完成后重启服务**

## 安装必要依赖

In [ ]:
!apt -y install -qq aria2
!pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.20 triton==2.0.0 -U
!pip install -q mediapipe==0.9.1.0 addict yapf fvcore omegaconf

## 创建工作区

In [ ]:
import os

!apt -y update -qq
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

!git clone https://github.com/comfyanonymous/ComfyUI /content/ComfyUI
%cd /content/ComfyUI
!pip install -q -r requirements.txt

# !git reset --hard
!git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors /content/ComfyUI/custom_nodes/comfy_controlnet_preprocessors
%cd /content/ComfyUI/custom_nodes/comfy_controlnet_preprocessors
!python install.py --no_download_ckpts
%cd /content/ComfyUI

!git clone https://github.com/JcandZero/ComfyUI_GLM4Node /content/ComfyUI/custom_nodes/ComfyUI_GLM4Node
%cd /content/ComfyUI/custom_nodes/ComfyUI_GLM4Node
!pip install -q -r requirements.txt
%cd /content/ComfyUI

!git clone https://github.com/ltdrdata/ComfyUI-Manager /content/ComfyUI/custom_nodes/ComfyUI-Manager
%cd /content/ComfyUI/custom_nodes/ComfyUI-Manager
# !pip install -q -r requirements.txt
%cd /content/ComfyUI

!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared-linux-amd64 && chmod 777 /content/cloudflared-linux-amd64
import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['/content/cloudflared-linux-amd64', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(8188, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/OpenPoseXL2.safetensors -d /content/ComfyUI/models/controlnet -o OpenPoseXL2.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-canny-rank128.safetensors -d /content/ComfyUI/models/controlnet -o control-lora-canny-rank128.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-canny-rank256.safetensors -d /content/ComfyUI/models/controlnet -o control-lora-canny-rank256.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-depth-rank128.safetensors -d /content/ComfyUI/models/controlnet -o control-lora-depth-rank128.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-depth-rank256.safetensors -d /content/ComfyUI/models/controlnet -o control-lora-depth-rank256.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-recolor-rank128.safetensors -d /content/ComfyUI/models/controlnet -o control-lora-recolor-rank128.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-recolor-rank256.safetensors -d /content/ComfyUI/models/controlnet -o control-lora-recolor-rank256.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-sketch-rank128-metadata.safetensors -d /content/ComfyUI/models/controlnet -o control-lora-sketch-rank128-metadata.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-sketch-rank256.safetensors -d /content/ComfyUI/models/controlnet -o control-lora-sketch-rank256.safetensors

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -d /content/ComfyUI/models/upscale_models -o RealESRGAN_x2plus.pth
base = "https://huggingface.co/ckpt/sd_xl_base_1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {base} -d /content/ComfyUI/models/checkpoints -o sd_xl_base_1.0.safetensors
refiner = "https://huggingface.co/ckpt/sd_xl_refiner_1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {refiner} -d /content/ComfyUI/models/checkpoints -o sd_xl_refiner_1.0.safetensors

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sdxl_vae/resolve/main/sdxl_vae.safetensors -d /content/ComfyUI/models/vae -o sdxl_vae.vae.safetensors


## 安装基本的库

In [ ]:
!git clone https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved /content/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved
%cd /content/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved
# !pip install -q -r requirements.txt
%cd /content/ComfyUI

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/119057 -d /content/ComfyUI/models/checkpoints -o meinamix_meinaV11.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt -d /content/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/models/ -o mm_sd_v15_v2.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v3_sd15_mm.ckpt -d /content/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/models/ -o v3_sd15_mm.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_ZoomIn.ckpt -d /content/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/ -o v2_lora_ZoomIn.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v3_sd15_adapter.ckpt -d /content/ComfyUI/models/loras/ -o v3_sd15_adapter.ckpt


In [ ]:
!git clone https://github.com/cubiq/ComfyUI_IPAdapter_plus /content/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus
%cd /content/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus
# !pip install -q -r requirements.txt
%cd /content/ComfyUI


!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors -d /content/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/models/ -o ip-adapter-plus_sd15.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors -d /content/ComfyUI/models/clip_vision/ -o model_v15.safetensors


In [ ]:
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/181356 -d /content/ComfyUI/models/checkpoints -o darkSushiMixMix_225D.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/333102 -d /content/ComfyUI/models/vae -o orangemix.vae.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/150193 -d /content/ComfyUI/models/loras -o edgBodytape_MINI.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/6056 -d /content/ComfyUI/models/embeddings/ -o bad-artist-anime.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/5637 -d /content/ComfyUI/models/embeddings/ -o ng_deepnegative_v1_75t.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/20170 -d /content/ComfyUI/models/embeddings/ -o bad-picture-chill-75v.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/125849 -d /content/ComfyUI/models/embeddings/ -o bad-hands-5.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/125849 -d /content/ComfyUI/models/embeddings/ -o bad-hands-5.pt

## 启动ComfyUI服务，请在使用Manager安装后，重启服务

### 服务地址为第一行打印内容，似乎里面的不太好用，还是在notebook里面手动吧

In [ ]:
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(8188, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)
!python main.py --dont-print-server
